## Why concurrency?

Processors have typically increased their frequency (i.e. number of operations per logical core) over time. However this has plateaued recently due to physical limitations.

Nowadays, we are more focused on increasing the number of logical cores in processors (multiplied by ~10x in recent years).

## Computer Architecture

Important to bear in mind that accessing RAM is much slower than a CPU cycle, accessing hard drive storage is much slower than accessing RAM, and network requests are even slower than that.

1 CPU can only run 1 process at a time.

Q: How did computer games work with single core computers?

A: The OS scheduler is able to context switch quickly between processes. Giving you the impression that things are running in parallel but actually not true parallelism (concurrency vs. parallelism)

## Multi-threading

Idea: spin up multiple workers in your processsor, assign some work to each of them and run them in parallel. E.g. we have 3 x 2s operations which would run serially in 6s or in parallel in 2s.

Use the `threading` module and the `Thread` class in Python.

Threads:
- All live in the same process.
- Can easily access shared data within the process they were spawned in.
- Are lightweight

*However*

This easy access to shared data comes with problems such as race conditions - where threads access and modify outdated data which leads to invalid transactions.

This can be resolved using thread synchronization such as locks, but this comes with its own set of complications such as the requirement for other developers to remember to use the lock and deadlocks, which is when each thread is waiting for another thread to release a lock and the program stops.

Summary: writing synchronised, multi-threaded code is hard and prone to errors.

## The Global Interpreter Lock (GIL)

The CPython implementation of Python contains a safety feature known as the Global Interpreter Lock. This prevents threads from running in parallel, to circumvent many of the issues that arise when sharing data across threads.

I/O bound tasks (e.g. processing an API request) don't require the CPU for most of the task time (see comparison of access times). Therefore they release the GIL and can take advantage of multithreading in Python. However, CPU bound tasks (such as matrix multiplications) will not be sped up by multithreading, in Python.

Note: this is a feature specifically of CPython and other languages can use multithreading to acheive parallelism in CPU bound tasks.

Aside: What about NumPy? NumPy is based on C libraries which are able to release the GIL and allows multithreading. This is why pure NumPy based operations are often much faster than their naive Python implementations - under the hood they can achieve parallelism with multiple threads. You can also multithread some NumPy functions that release the GIL using Python.

## Multiprocessing

So, how do we create parallelism in Python for CPU bound tasks?

We can use multi-processing.

Multiprocessing:
- Is slower and more resource heavy than multithreading
- Doesn't share data as easily as multithreading

When multiprocessing, we spawn a new process for every task, which has its own data, memory, file descriptors, etc. By default the data is copied from the original process.

However, there is no shared data by default and so we overcome the GIL.

In Python, we can use the `multiprocessing` module and the `Process` class

In [1]:
import multiprocessing as mp
from multiprocessing import Process
import random
import time

In [2]:
mp.set_start_method('fork')

In [3]:
def is_prime(n):
    if n in (2, 3):
        return True
    if n % 2 == 0:
        return False
    for divisor in range(3, n, 2):
        if n % divisor == 0:
            return False
    return True

In [4]:
def check_prime_worker(number):
    if is_prime(number):
        print(f'{number} is prime', flush=True)
    else:
        print(f'{number} is not prime', flush=True)

In [5]:
numbers = [15492781, 15492787, 15492803, 15492811, 15492810, 15494819, 15494623, 15495941, 15492859, 15527509]

In [6]:
[check_prime_worker(n) for n in numbers];

15492781 is prime
15492787 is prime
15492803 is prime
15492811 is prime
15492810 is not prime
15494819 is not prime
15494623 is prime
15495941 is prime
15492859 is prime
15527509 is prime


In [7]:
processes = [Process(target=check_prime_worker, args=(n,)) for n in numbers]

In [8]:
start = time.time()

In [9]:
[p.start() for p in processes];

15492810 is not prime
15494819 is not prime
15527509 is prime15495941 is prime15492803 is prime


15492781 is prime15492859 is prime15492811 is prime

15492787 is prime
15494623 is prime



In [10]:
[p.join() for p in processes];

In [11]:
time.time() - start

2.7395849227905273

In [12]:
[p.close() for p in processes];

Note that here we were not able to share data across processes and instead resorted to printing out the results which is clearly not ideal for a real program.

One option to share data between processes is to use queues, which can be used to pull in tasks and add the results of completing the task. 

The `Pool` class simplifies this and allows some data sharing to occur 'behind the scenes' using the `map` and `map_async` methods. Using a context manager also takes care of joining and closing all the processes in the pool.

In [13]:
start = time.time()

In [14]:
with mp.Pool(processes=4) as pool:
    results = pool.map(is_prime, numbers)

In [15]:
time.time() - start

2.681166172027588

In [16]:
f"Found {sum(results)} primes out of the {len(results)} numbers"

'Found 8 primes out of the 10 numbers'

A limitation of the multiprocessing `map` method is that you can only pass a single variable argument to the function being passed to the pool. To overcome this limitation you can instead use `starmap`.

In [17]:
def is_prime(a, b):
    n = a + b
    if n in (2, 3):
        return True
    if n % 2 == 0:
        return False
    for divisor in range(3, n, 2):
        if n % divisor == 0:
            return False
    return True

In [18]:
numbers = [(15492780, 1), 
           (15492780, 7), 
           (15492799, 104), 
           (15492800, 11), 
           (15492800, 10), 
           (15494810, 9), 
           (15494619, 4), 
           (15495938, 3), 
           (15492858, 1), 
           (15527507, 2)]

In [19]:
with mp.Pool(processes=4) as pool:
    results = pool.starmap(is_prime, numbers)

In [20]:
results

[True, True, False, True, False, False, True, True, True, True]

You can also use `functools.partial` with the standard `map` method. (But it seems like only if the additional arguments are fixed?)

In [21]:
from functools import partial

In [22]:
numbers = [15492781, 15492787, 15492803, 15492811, 15492810, 15494819, 15494623, 15495941, 15492859, 15527509]
B = 2

In [23]:
with mp.Pool(processes=4) as pool:
    results = pool.map(partial(is_prime, b=B), numbers)

## `concurrent.futures`

The `concurrent.futures` library is a relatively recent addition to the standard library that provides a high-level API for multithreading and multiprocessing.

In [24]:
import concurrent.futures as cf
from concurrent.futures import ProcessPoolExecutor

In [25]:
def is_prime(n):
    if n in (2, 3):
        return True
    if n % 2 == 0:
        return False
    for divisor in range(3, n, 2):
        if n % divisor == 0:
            return False
    return True

In [26]:
numbers = [15492781, 15492787, 15492803, 15492811, 15492810, 15494819, 15494623, 15495941, 15492859, 15527509]

The entry points to the library are  PoolExecutors (which can be a `ThreadPoolExecutor` or a `ProcessPoolExecutor`, the APIs are identical)

A similar map method is defined

In [27]:
with ProcessPoolExecutor(max_workers=4) as ex:
    results = ex.map(is_prime, numbers)

In [28]:
list(results)

[True, True, True, True, False, False, True, True, True, True]

To pass multiple arguments to the multiprocessed/threaded function, we can use the `submit` and `as_completed` pattern.

In [29]:
def is_prime(a, b):
    n = a + b
    if n in (2, 3):
        return True
    if n % 2 == 0:
        return False
    for divisor in range(3, n, 2):
        if n % divisor == 0:
            return False
    return True

In [30]:
numbers = [(15492780, 1), 
           (15492780, 7), 
           (15492799, 104), 
           (15492800, 11), 
           (15492800, 10), 
           (15494810, 9), 
           (15494619, 4), 
           (15495938, 3), 
           (15492858, 1), 
           (15527507, 2)]

In [31]:
with ProcessPoolExecutor(max_workers=4) as ex:
    futures = [ex.submit(is_prime, a, b) for a, b in numbers]
    results = [future.result() for future in cf.as_completed(futures)]

In [32]:
results

[False, False, False, True, True, True, True, True, True, True]

Further investigation:

- What about Numba?
- What about Dask?
- What about Ray?
- What about Cython?